# Módulo Deep Learning y computer vision - practica final

Jose Del Hierro

1. Carga de datos
2. Preprocesado de los datos
3. Hito 1: Modelos 1D
4. Hito 2: Modelos 2D
5. Hito 3: Late-fusion
6. Hito 4: Early-fusion
7. Resultados
8. Discusión

## 1. Carga de los datos

En primer lugar debemos cargar las librerías necesarias para interactuar con datos que tengamos en local, o en Google Drive.

In [ ]:
# permitimos el acceso a nuestra carpeta de GDrive para simplificar el proceso,
# si bien podríamos subir manualmente los datos a mano.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas
from pathlib import Path

DATA_DIR = "/content/drive/MyDrive/deep_learning-main/practica"
METADATA = "HAM10000_metadata.csv"
IMAGES = "hnmist_28_28_RGB.csv"

def load_dataset(N: int = None):
    """ Load the dataset up to `N` samples. If `N` is None, load the entire dataset. """
    metadata = pandas.read_csv(Path(DATA_DIR) / METADATA)
    images = pandas.read_csv(Path(DATA_DIR) / IMAGES)
    if N is not None:
        metadata = metadata[:N]
        images = images[:N]
    return metadata, images

tab_data, im_data = load_dataset()

Visualizamos las primeras muestras:

In [ ]:
tab_data.head(10)

In [ ]:
im_data.head(10)

## 2. Preprocesado de los datos

En primer lugar, eliminamos aquellas columnas de metadatos que no aporten información útil a nuestro proceso de modelado. Igualmente, identificamos la columna `dx` como nuestra columna de etiquetas. Por comodidad en el futuro, la renombramos para que no haya lugar a confusiones más adelante.

In [ ]:
tab_data = tab_data.drop(columns=['lesion_id', 'image_id'])
# Se elimina lesion_id e image_id porque no nos sirven
tab_data.rename(columns={'dx': 'label'}, inplace=True)
tab_data.head(10)

In [ ]:
tab_data.columns

In [ ]:
for col in tab_data.columns:
    print(col, tab_data[col].unique())

Hay valores de "Unknown", vamos a ver cuantos son para determinar que hacer mejor

In [ ]:
def unknown_stats(col):
    total = len(tab_data)
    unknown_count = (tab_data[col] == "unknown").sum()
    percentage = unknown_count / total * 100
    print(f"{col.upper():<15}: {unknown_count} unknowns ({percentage:.2f}%)")

unknown_stats("sex")
unknown_stats("localization")


Eliminamos los valores de "Unknown" porqe no representan ni el 5 % de los valores y al ser categóricos no se puede imputar directamente

In [ ]:
tab_data = tab_data[tab_data["sex"] != "unknown"]
tab_data = tab_data[tab_data["localization"] != "unknown"]

In [ ]:
for col in tab_data.columns:
    print(col, tab_data[col].unique())

 **Revisando valores nulos**

In [ ]:
(tab_data.isnull().mean() * 100).round(2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=tab_data["age"])
plt.title("Boxplot de Age")
plt.show()

In [ ]:
tab_data["age"].min(), tab_data["age"].max()

**Al ser una variable de edad es muy posible que este valor del outlier sea real (0)**

Llenamos los valores nulos con la mediana porque hay outliers

In [ ]:
median_age = tab_data["age"].median()

tab_data["age"] = tab_data["age"].fillna(median_age)

In [ ]:
(tab_data.isnull().mean() * 100).round(2)

**Volviendo las categóricas numéricas**

In [ ]:
tab_col_translation = {}
dx_type = tab_data['dx_type']
for col in ["label", "dx_type", "sex"]:
    factorized = pandas.factorize(tab_data[col])
    tab_data[col] = factorized[0]
    tab_col_translation[col] = factorized[1]
tab_data.head(10)

In [ ]:
tab_col_translation

One hot encoding a "localization"

In [ ]:
localizations = pandas.get_dummies(tab_data['localization']).astype(int)
tab_data = pandas.concat(
    [
        tab_data.drop(columns=['localization']),
        localizations
    ],
    axis=1
)
tab_data.head(10)

La columna `dx_type` no se vislumbra muy bien si nos pueda servir, por ello vamos a revisar la correlación que esta pueda tener con `label`

In [ ]:
tab_data[['label', 'dx_type']].corr()

Separamos las `features` de manera explícita

In [ ]:
labels = tab_data['label']
tab_data = tab_data.drop(columns=['label', 'dx_type'])

Ahora, procedemos con la normalización de las imágenes.(2D)

In [ ]:
im_data = im_data / 255.0
im_data.head(10)

Seguimos ya con los datos listos a la división de los mismos entre tran test y validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tab_data, labels, test_size=0.3, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

Convertimos la edad en números acotados entre `[0, 1]`, imputando valores que están missing. Alternativamente, eliminamos esas muestras (en cuyo caso tendremos que eliminar también las imágenes asociadas del dataset):

In [ ]:
import seaborn as sns
sns.boxplot(X_train['age'])


In [ ]:
X_train['age'].hist()

In [ ]:
from sklearn.preprocessing import StandardScaler

age_scaler = StandardScaler()
X_train['age'] = age_scaler.fit_transform(X_train[['age']])
X_val['age']   = age_scaler.transform(X_val[['age']])
X_test['age']  = age_scaler.transform(X_test[['age']])


En el análisis exploratorio Se descubrieron algunas columnas con datos o desconocidos o nulos. al ser estos menores al 5% de cada columna, opté por mejor eliminarlos y no ingresar datos artificiales al dataset.
Después de esto procedí a normalizar la variable age con StandardScaler para hacer el modelo más estable.

Separamos ahora las imágenes

In [ ]:
im_train = im_data.loc[X_train.index]
im_val = im_data.loc[X_val.index]
im_test = im_data.loc[X_test.index]

Como trabajamos sobre imágenes RGB, escalaremos cada canal por separado. Por comodidad, convertiremos las imágenes que estan actualmente aplanadas, a su representación 2D (28 x 28) para luego volver a aplanarlas y asi no se pierdan los colores.

In [ ]:
im_scaler = [StandardScaler() for _ in range(3)]
batched_imgs_train = im_train.values.reshape(-1, 28*28, 3)
batched_imgs_val = im_val.values.reshape(-1, 28*28, 3)
batched_imgs_test = im_test.values.reshape(-1, 28*28, 3)
for c, channel in enumerate(['R', 'G', 'B']):
    im_channel = batched_imgs_train[..., c]
    im_scaler[c].fit(im_channel)
    batched_imgs_train[..., c] = im_scaler[c].transform(im_channel)
    batched_imgs_val[..., c] = im_scaler[c].transform(batched_imgs_val[..., c])
    batched_imgs_test[..., c] = im_scaler[c].transform(batched_imgs_test[..., c])

print(f"Inmediatamente post-normalización: {batched_imgs_train.shape}")

im_train = batched_imgs_train.reshape(-1, 28 * 28 * 3)
im_val = batched_imgs_val.reshape(-1, 28 * 28 * 3)
im_test = batched_imgs_test.reshape(-1, 28 * 28 * 3)
print(f"Vuelta a versión aplanada: {im_train.shape}")


Finalmente, convertimos las etiquetas a su representación *one-hot*.

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
y_train_ = to_categorical(y_train)
y_val_ = to_categorical(y_val)
y_test_ = to_categorical(y_test)

## 3. Hito 1: Modelos 1D

Importaciones previas

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


Definimos la función para entrenar nuestra red con los parámetros deseados

In [ ]:
def train_tab_model(activation_function, learning_rate, batch_size, num_epochs):

    # Entrada:
    input_tab = Input(shape=(X_train.shape[1],))

    x = Dense(100, activation=activation_function)(input_tab)
    x = Dense(64, activation=activation_function)(x)
    x = Dense(32, activation=activation_function)(x)
    # Salida
    output_tab = Dense(y_train_.shape[1], activation="softmax")(x)

    model = Model(inputs=input_tab, outputs=output_tab)

    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

    print("[INFO]: Entrenando red tabular...")
    # Entrenando la solución
    H = model.fit(X_train, y_train_,validation_data=(X_val, y_val_),epochs=num_epochs,batch_size=batch_size,verbose=1)

    # Evaluando el modelo de predicción con las imágenes de test
    print("[INFO]: Evaluando red tabular...")
    preds = model.predict(X_test, batch_size=batch_size)
    print(classification_report(y_test_.argmax(axis=1), preds.argmax(axis=1)))

    # Curvas de entrenamiento
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.plot(H.history["accuracy"], label="train_acc")
    plt.plot(H.history["val_accuracy"], label="val_acc")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Loss/Accuracy")
    plt.title("Hito 1 - Modelo tabular")
    plt.show()

    return model, H


In [ ]:
act = tf.nn.relu
learning_rate = 0.01
num_epochs = 10
batch_size = 128
train_tab_model(act, learning_rate, batch_size, num_epochs)

Empece con un Learning rate muy alto (0.1) para Adam, al utilizar (0.01) mejoró.
Se puede observar que la red está aprendiendo patrones y no hay un sobre ajuste (val_accuracy muy similar a accuracy).

Se puede explicar "algo" de la lesión con datos demográficos, sin embargo sin información visual no se puede confirmar de todo el tipo de lesión.


  Si bien ya sabemos que siempre hay margen de mejora, este parece sin duda un modelo bastante prometedor.

## 4. Hito 2: Modelos 2D

Aunque se puede diseñar una red neuronal desde cero, al tener un dataset de aproximadamente 10k imágenes, nos conviene usar un modelo preentrenado y realizar un fine tuning con algunas capas del backbone para adaptarlo bien a nuestro dataset. Enn este caso se usará EfficientNetBO y se descongelarán las últimas 10 capas por la cantidad de imágenes en HAM10000. Al tocar las capas profundas, corremos menos riesgo de que el modelo aprenda demasiado de nuestro dataset pero se liberan las capas finales, en las cuales estan los detalles más finos y los que en realidad necesita aprender nuestro modelo.

In [ ]:
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

input_shape = (224, 224, 3)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

# resize train set
X_train_resized = []
for img in X_train:
  X_train_resized.append(np.resize(img, input_shape) / 255)

X_train_resized = np.array(X_train_resized)
print(X_train_resized.shape)

# resize test set
X_test_resized = []
for img in X_test:
  X_test_resized.append(np.resize(img, input_shape) / 255)

X_test_resized = np.array(X_test_resized)
print(X_test_resized.shape)

Vamos a utilizar EfficientNetB0 debido a su pequeño tamaño, y por tanto su velocidad. Se debe redimensionar las imagenes ya que este modelo tiene un input de otro tamaño

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Resizing, GlobalAveragePooling2D

input_tensor = Input(shape=(28, 28, 3))
x = Resizing(224, 224)(input_tensor)
x = EfficientNetB0(weights='imagenet', include_top=False)(x) # Quitamos la "cabeza" del modelo para poder entrenar el nuestro con pocas imagenes
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dropout(0.3)(x) #  Debemos mitigar el overfitting en las capas densas de EfficientNetB0
x = Dense(64, activation='relu', name='fc3')(x)
x = Dropout(0.2)(x)
output_tensor = Dense(y_train_.shape[1], activation='softmax', name='cls_head')(x)
vision = Model(inputs=input_tensor, outputs=output_tensor)

vision.summary()

In [ ]:
im_train = im_train.reshape(-1, 28, 28, 3)
im_val = im_val.reshape(-1, 28, 28, 3)
im_test = im_test.reshape(-1, 28, 28, 3)

Congelamos el modelo principal a excepción de las últimas capas para hacer fine tuning con nuestras imágenes del dataset.

In [ ]:
for layer in vision.layers:
  if layer.name not in ['fc1', 'fc2', 'cls_head']:
    layer.trainable = False

N = 10
for layer in vision.layers[-N:]:
    layer.trainable = True
vision.summary()


El batch size por defecto de keras es 32 y el learning rate de adam = 0.001 se dejaron estos valores para no exceder la memoria disponible

In [ ]:
  # compilamos
  vision.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # definimos el early-stopping
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=0.005)

  # entrenamos
  im_train.shape
  H = vision.fit(im_train, y_train_, validation_data=(im_val, y_val_), epochs=15, callbacks=[early_stopping])

In [ ]:
fig, ax = plt.subplots()
ax.plot(H.history['loss'], label='train')
ax.plot(H.history['val_loss'], label='val')
ax.legend()
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.show()

## 5. Hito 3: late-fusion

Combinamos las predicciones de imágenes y clasificación

In [ ]:
def build_late_fusion_model():
    # Consideramos los mismos inputs que cada modelo por separado
    tabular_input = Input(shape=tabular_input_shape, name="tabular_input")
    vision_input = Input(shape=vision_input_shape, name="vision_input")

    # Obtenemos las predicciones finales para cada modalidad
    tabular_pred = tabular_model(tabular_input)
    vision_pred = vision_model(vision_input)

    # Fusión de las probabilidades concatenadas
    merged = Concatenate()([tabular_pred, vision_pred])

    # Clasificador final
    output = Dense(3, activation="softmax", name="final_output")(merged)

    # Definición final del modelo con nuevo classificador al final
    late_fusion_model = Model(inputs=[tabular_input, vision_input], outputs=output)
    return late_fusion_model

model = build_late_fusion_model()
model.summary()

In [ ]:
  # compilamos
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # 2. Entrenar
history = model.fit([X_tab_train, X_img_train],y_train_,validation_data=([X_tab_val, X_img_val], y_val_),epochs=20,batch_size=32)
    # epochs es bajo porque no estamos entrenando desde cero.

# 3. Evaluar
test_loss, test_acc = model.evaluate([X_tab_test, X_img_test],y_test_)

print("Accuracy Late Fusion:", test_acc)

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.title("Late Fusion - Loss y Accuracy")
plt.xlabel("Época")
plt.show()

## 6. Hito 4: Early-fusion

En este punto extraeremos los embeddings extraídos tras las capas convolucionales en el caso de las imágenes, mientras que tomaremos la representación de 32 características de la primera capa fully-connected del modelo tabular.

Early Fusion con clasificador keras

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping

# Shapes de tus entradas
tabular_input_shape = X_train.shape[1:]      # Ej: (n_features,)
vision_input_shape  = im_train.shape[1:]     # (28, 28, 3)
num_classes = y_train_.shape[1]

def build_early_fusion_model(tabular_model, vision_model):

    # 1. Entradas
    tabular_input = Input(shape=tabular_input_shape, name="tabular_input")
    vision_input  = Input(shape=vision_input_shape,  name="vision_input")

    # 2. Extraemos las FEATURES de cada rama
    tabular_intermediate = Model(
        inputs=tabular_model.input,
        outputs=tabular_model.get_layer("tabular_features").output,
        name="tabular_feat_extractor"
    )

    vision_intermediate = Model(
        inputs=vision_model.input,
        outputs=vision_model.get_layer("vision_features").output,
        name="vision_feat_extractor"
    )

    # 3. Pasamos ambas entradas por sus extractores
    tab_features = tabular_intermediate(tabular_input)
    vis_features = vision_intermediate(vision_input)

    # 4. Fusión temprana
    merged = Concatenate(name="fused_features")([tab_features, vis_features])

    # 5. Clasificador conjunto
    x = Dense(128, activation="relu", name="fusion_fc1")(merged)
    x = Dense(64,  activation="relu", name="fusion_fc2")(x)
    output = Dense(num_classes, activation="softmax", name="final_output")(x)

    # 6. Modelo final
    fusion_model = Model(inputs=[tabular_input, vision_input],outputs=output,name="early_fusion_model")

    return fusion_model


# Construimos el modelo
early_fusion_model = build_early_fusion_model(tabular_model, vision_model)

early_fusion_model.summary()

# Compilamos
early_fusion_model.compile(optimizer=Adam(learning_rate=1e-4),loss="categorical_crossentropy",metrics=["accuracy"])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss',patience=10,
restore_best_weights=True,min_delta=0.005)

# Entrenamiento
H_fusion = early_fusion_model.fit([X_train, im_train], y_train_,validation_data=([X_val, im_val], y_val_),epochs=50,batch_size=32,callbacks=[early_stopping],verbose=1)

# Evaluación en test
test_loss, test_acc = early_fusion_model.evaluate([X_test, im_test], y_test_, verbose=0)
print("Test accuracy (Early fusion Keras):", test_acc)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

plt.plot(H_early.history["accuracy"], label="Train Accuracy", linewidth=2)
plt.plot(H_early.history["val_accuracy"], label="Validation Accuracy", linewidth=2)

plt.plot(H_early.history["loss"], label="Train Loss", linestyle="--", linewidth=2)
plt.plot(H_early.history["val_loss"], label="Validation Loss", linestyle="--", linewidth=2)

plt.title("Training Curves – Early Fusion Model", fontsize=16)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Accuracy / Loss", fontsize=14)
plt.grid(alpha=0.3)
plt.legend(fontsize=12)
plt.show()


Se puede apreciar que el modelo puede aprender de relaciones que por separado en cada data frame no se podrian dar. el modelo early fusion mostró mayor aprovechamiento de la información. en general es interesante la idea de utilizar dos modelos que aparentemente son demasiado distintos.

## 7. Resultados

Después de completar el desarrollo de los distintos modelos(tabular, vision, late fusion, etc.) y habiendo validado los mismos. Se puede decir que el rendimiento es adecuado para continuar con la evaluación final. Los modelos ya muestran un comportamiento estable y coherente.

### 7.0. ZeroR (baseline)


In [ ]:
counts = y_test.value_counts()
counts.index = [tab_col_translation['label'][i] for i in counts.index]
print(counts)

### 7.1. Datos tabulares


In [ ]:
from sklearn.metrics import classification_report

tab_preds = tab_model.predict(X_test.values)
gt_test = numpy.argmax(y_test_, axis=1)
cr = classification_report(gt_test, numpy.argmax(tab_preds, axis=1), target_names=tab_col_translation['label'])
print(cr)

### 7.2. Imágenes

In [ ]:
vis_preds = vision.predict(im_test)
cr = classification_report(gt_test, numpy.argmax(tab_preds, axis=1), target_names=tab_col_translation['label'])
print(cr)

### 7.3. Late-fusion

In [ ]:
late_fusion_preds = clf.predict(numpy.concatenate([tab_preds_test, im_preds_test], axis=1))
cr = classification_report(gt_test, late_fusion_preds, target_names=tab_col_translation['label'])
print(cr)

### 7.4. Early-fusion

In [ ]:
test_inputs = {
    "tabular_input": X_test,
    "vision_input": im_test
}

early_fusion_preds = early_fusion_model.predict(test_inputs)
cr = classification_report(gt_test, numpy.argmax(early_fusion_preds, axis=1), target_names=tab_col_translation['label'])
print(cr)

### 7.5. Estudio por método de diagnóstico

In [ ]:
print(f"Classes correspond to: {list(tab_col_translation['label'])}")
dx_methods = dx_type[X_test.index].unique()
for method in dx_methods:
  is_method = (dx_type[X_test.index] == method).values
  method_labels = y_test[is_method]
  method_predictions = numpy.argmax(tab_preds[is_method], axis=1)
  print(f"Method: {method}")
  cr = classification_report(method_labels, method_predictions)
  print(cr)


### 7.6. Estudio por sexo

In [ ]:
sex = tab_data["sex"][X_test.index].unique()
sex_str = ["male", "female", "unknown"]
for s in sex:
  is_sex = (tab_data["sex"][X_test.index] == s).values
  sex_labels = y_test[is_sex]
  sex_predictions = numpy.argmax(tab_preds[is_sex], axis=1)
  print(f"Sex: {sex_str[s]}")
  cr = classification_report(sex_labels, sex_predictions)
  print(cr)

### 7.8. Interpretación modelo tabular (SHAP)

In [ ]:
import shap

dx_names = list(tab_col_translation["label"])
explainer = shap.DeepExplainer(tab_model, X_test.to_numpy())
shap_values = explainer.shap_values(X_test.to_numpy())
i = 1
shap_values_class_i = shap_values[i]
shap.summary_plot(shap_values_class_i, feature_names=X_test.columns, plot_type="bar")

### 7.9. ROC-AUC

Esta métrica, el *Receiver Operating Curve - Area Under the Curve*  es especialmente útil para valorar el funcionamiento de un clasificador binario, a nivel de la calidad de las probabilidades predichas.

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Debe realizarse para clases por separado, ya que es una métrica diseñada para escenarios binarios.
def compute_and_plot_AUROC(y_true, y_probs, label_names):
  for i, label_name in enumerate(label_names):
    gt = (y_true == i).astype(numpy.int8)
    label_preds = y_probs[:, i]
    fpr, tpr, thresholds = roc_curve(gt, label_preds)
    roc_auc = roc_auc_score(gt, label_preds)
    plt.plot(fpr, tpr, label=f'[{label_name}] AUC = {roc_auc:.2f})', alpha=0.5)
  plt.plot([0, 1], [0, 1], 'k--', label='Random')  # La diagonal representa un clasifiador aleatorio
  plt.xlabel('False Positive Rate (1 - Specificity)')
  plt.ylabel('True Positive Rate (Sensitivity)')
  plt.legend(loc='lower right')
  plt.show()


compute_and_plot_AUROC(y_test, early_fusion_preds, tab_col_translation["label"])



## 8. Discusión

En la práctica todos los modelos ofrecen resultados razonables para una primera aproximación sin negar las limitaciones de los mismos.

La combinación de ambos modelos permitió reducir varias de las limitaciones. Es importante seguir explorando para poder mejorar más el modelo y refinar la arquitectura de la cabeza del modelo.
En la práctica pude evidenciar cómo la eficiencia se vuelve un factor crucial al momento de poser entrenar a un modelo y sobre todo cuando usas un backbone. Tener más capas o neuronas muchas veces representa más un riesgo de overfitting y consumo de memoria RAM que necesariamente una mejora del modelo.